In [1]:
import numpy as np
from sklearn.linear_model import LogisticRegression#회귀 분류
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import VotingClassifier# fier 분류/ 앙상블형태의 

In [2]:
X = np.load('day7_data2_X.npy')
y = np.load('day7_data2_y.npy')

In [3]:
c1 =LogisticRegression(random_state=1)
c2 = DecisionTreeClassifier(random_state=1)#max_depth 사전 가지치기,,max_depth=4
c3 = GaussianNB()
ec = VotingClassifier(estimators=[('lr',c1),('rf',c2),('gnb',c3)],voting='hard')#estimators= 리스트형으로서 모델들을 분류기에 하나씩 넣어줌

In [4]:
#테스트하기 위해 교차검증을 진행한다. 교차검증은 n가지중 젤 효율 좋은 값을 찾아낸다.
from sklearn.model_selection import cross_val_score
cross_val_score(ec,X,y,cv=5).mean()#교차검증 점수, cv몇가지로 분류해서 나타내겠는가
#앙상블은 절대적이지 않다. 단일 모델동작에서보다 좋지않을때도 있다.

0.8009268075922046

In [5]:
cross_val_score(c1,X,y,cv=5).mean()

0.8290420872214816

In [6]:
cross_val_score(c2,X,y,cv=5).mean()

0.7829175395162826

In [7]:
cross_val_score(c3,X,y,cv=5).mean() #성능이 젤 떨어짐

0.4600139655938551

In [8]:
#앙상블은 최적적의 선택을하기때문에 오래걸린다.
#회구문제에서 가중평균이나 단순평균을 구하는방식으로 y 예측 => 쓰레기 결과도 최종결과에 영향을 줄수도있다.
ec1 = VotingClassifier(estimators=[('lr',c1),('dt',c2)],voting ='hard')
cross_val_score(ec1,X,y,cv=5).mean()                                                   

0.8222687742017394

In [9]:
#LogisticRegression(C=,penalty=,solver=,)#로지스틱회귀 파라미터
c_params =  [0.1,5.0,7.0,10.0,15.0,20.0,100.0]
params = {
    'lr__solver':['liblinear'],
    'lr__penalty':['l2'],#리스트를 씌워서 extend 구조를 보내야하기때문에 이렇게보낸다.
    'lr__C':c_params,
    'dt__criterion':['gini','entropy'], #엔트로피는 종류가 너무 많을때 정보이득값이 높게 잡힐때도있다.
    'dt__max_depth':[10,8,7,6,5,4,3,2],
    'dt__min_samples_leaf':[1,2,3,4,5,6,7,8,9]
}

In [10]:
from sklearn.model_selection import GridSearchCV
g=GridSearchCV(estimator=ec1,param_grid=params,cv=5)
g.fit(X,y)
g.best_score_


0.8425569732749316

In [11]:
g.best_params_

{'dt__criterion': 'gini',
 'dt__max_depth': 10,
 'dt__min_samples_leaf': 5,
 'lr__C': 5.0,
 'lr__penalty': 'l2',
 'lr__solver': 'liblinear'}

In [ ]:
ec2 = VotingClassifier(estimators=[('A',c1),('B',c2)],voting ='hard')
params = {
    'A__solver':['liblinear'],
    'A__penalty':['l2'],#리스트를 씌워서 extend 구조를 보내야하기때문에 이렇게보낸다.
    'A__C':[5.0],
    'B__criterion':['gini'], #엔트로피는 종류가 너무 많을때 정보이득값이 높게 잡힐때도있다.
    'B__max_depth':[10],
    'B__min_samples_leaf':[5]
}
g1=GridSearchCV(estimator=ec2,param_grid=params,cv=5)
#g.best_score_

각각 개별로

In [13]:
c1 =LogisticRegression(solver='liblinear',penalty='l2',C=5.0,random_state=1)
c2 = DecisionTreeClassifier(criterion='gini',max_depth=10,min_samples_leaf=5,random_state=1)
ec3 = VotingClassifier(estimators=[('A',c1),('B',c2)],voting ='hard')
cross_val_score(ec3,X,y,cv=5).mean()

0.8425569732749316

정리조합

In [14]:
import numpy as np#파일 로드
from sklearn.linear_model import LogisticRegression#모델1
from sklearn.tree import DecisionTreeClassifier#모델2
from sklearn.ensemble import VotingClassifier#모델 조합
from sklearn.model_selection import cross_val_score#성능평가
#data로드
X = np.load('day7_data2_X.npy')
y = np.load('day7_data2_y.npy')
#모델 생성(최적파라미터 값을 적용)
c1 =LogisticRegression(solver='liblinear',penalty='l2',C=5.0,random_state=1)
c2 = DecisionTreeClassifier(criterion='gini',max_depth=10,min_samples_leaf=5,random_state=1)
#앙상블 모델 생성
ec = VotingClassifier(estimators=[('c1',c1),('c2',c2)],voting ='hard')
#검증
cross_val_score(ec,X,y,cv=5).mean()

0.8425569732749316